In [86]:
import numpy as np
import pandas as pd
from numpy import linalg as LA
from skimage import io, transform
import matplotlib.pyplot as plt
%matplotlib inline

In [87]:
#data=pd.read_csv('MESA_data_9f_PoolCohort.csv',sep=',')
data=pd.read_csv('MESA_data_9f_PoolCohort.csv',sep=',')

In [88]:
print(data.shape)
data=data.dropna()
print(data.shape)

(6459, 10)
(6454, 10)


In [89]:

X=np.array(data)[:,:-1]
Y=np.array(data)[:,-1]
print(X.shape)
print(Y.shape)
mean=X.mean(axis=0)
components=X.shape[1]
data_centered = X - X.mean(axis=0)

(6454, 9)
(6454,)


In [90]:
cov_mat = np.cov(data_centered, rowvar=False)
U,S,Vt = np.linalg.svd(cov_mat)

new_Vt=Vt.copy()
new_U=U.copy()
new_S=S.copy()


new_U=U[:,:components]
new_S=S[:components]
new_Vt=Vt[:components,:]

pc=new_U
scaled_singular_vals = np.zeros((components,1))

pc_projection=np.dot(data_centered,pc)
reconstruct=np.dot(pc_projection,Vt)+mean


In [91]:
def reconstruct_original(data,matrix):
    return np.dot(data.reshape(1,-1),matrix)

augmented_data=np.empty(shape=(0,components))
augmented_Ys=np.empty(shape=(0,1))
for i in range(pc_projection.shape[0]):
    datapoint=pc_projection[i]
    augmented_data=np.vstack((reconstruct_original(datapoint,Vt),augmented_data))
    augmented_Ys=np.vstack((Y[i],augmented_Ys))
    if(Y[i]==0):
        continue
    for k in range(11):
        for c in range(components):
            scaled_singular_vals[c][0] = np.random.normal(0, 0.2)*new_S[c]
        scaled_singular_vecs = np.dot(new_U,scaled_singular_vals)
        reconstruct=datapoint.reshape(1,-1)+scaled_singular_vecs.reshape(1,-1)
        #reconstruct=np.dot(reconstruct.T,Vt)
        augmented_data=np.vstack((reconstruct_original(reconstruct,Vt),augmented_data))
        augmented_Ys=np.vstack((1,augmented_Ys))

    augmented_data=np.dot(augmented_data,Vt)+mean
augmented_data=np.concatenate((augmented_data,augmented_Ys),axis=1)

print(augmented_data.shape)

(11734, 10)


In [92]:
np.savetxt('augmented.csv',augmented_data,delimiter=',')

In [93]:
5760+5974

11734